In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import json
import requests
import srsly
import pathlib

In [ ]:
# Code for grabbing variant props for covariants github
# Data from covariants cluster tables https://github.com/hodcroftlab/covariants/tree/master/cluster_tables, covariants.org

In [ ]:
# Point to path to variant data folder in data folder
path = pathlib.Path('../data/variant-data')

In [ ]:
# specify variant dictionary
variant_data_dict = {}
temp_dict = {}

# for loop to read through data paths in folders, grab json files and add them to dictionary
for p in path.iterdir():
    variant_string = p.stem
    variant_name = variant_string.split('_')[0].replace('.','_')
    variant_dict = srsly.read_json(p)
    variant_data_dict[variant_name] = variant_dict

In [ ]:
# check each variant dict key has been added
keysList = list(variant_data_dict.keys())
print(keysList)
# Note names are nextstrain calde names at this point and WHO VOC naming

In [ ]:
# create dataframe from nested variant dictionary
df = pd.DataFrame()

for variant,country in variant_data_dict.items():
    for c in country:
        new_df = pd.DataFrame(country[c])
        new_df['country'] = c
        new_df['nextclade_variant'] = variant
        df = pd.concat([df, new_df])
        
# create new variant proportion column
df['variant_prop'] = df['cluster_sequences']/df['total_sequences']

In [ ]:
# lets creare a new column that adds more common variant names

In [ ]:

# Define the function to categorise names
def variant_rename(variant_name):
    if variant_name == '20H_Beta_V2':
        return 'Beta'
    elif variant_name == '20I_Alpha_V1':
        return 'Alpha'
    elif variant_name == '21A_Delta':
        return 'Delta_1'
    elif variant_name == '21I_Delta':
        return 'Delta_2'
    elif variant_name == '21J_Delta':
        return 'Delta_3'
    elif variant_name == '21K_Omicron':
        return 'Omicron_BA1'
    elif variant_name == '21L_Omicron':
        return 'Omicron_BA2'
    elif variant_name == '22A_Omicron':
        return 'Omicron_BA4'
    else:
        return 'Omicron_BA5'

# Apply the function to the Age column using the apply() function
df['variant'] = df['nextclade_variant'].apply(variant_rename)

In [ ]:
df

In [ ]:
# select countries
countries = ['Malaysia', 'Philippines', 'Vietnam']

df_analysis = df.loc[df['country'].isin(countries)]

In [ ]:
# identify week first time proportion goes above a threshold
threshold = 0.01

emergence_df = df_analysis.loc[df_analysis.variant_prop>threshold].groupby(['country','variant']).head(2)

In [ ]:
emergence_df.loc[emergence_df['country'] == 'Malaysia']

In [ ]:
## some old code 
# Combine variant dictionaries into single nested dictionary
variant_data_dict = {'Beta_20H':Beta_20H,
                    'Alpha_20I':Alpha_20I,
                     'Delta_21A':Delta_21A,
                     'Delta_21I':Delta_21I,
                     'Delta_21J': Delta_21J,
                     'Omicron_21K_BA1': Omicron_21K_BA1,
                     'Omicron_21L_BA2': Omicron_21L_BA2,
                     'Omicron_22A_BA4': Omicron_22A_BA4,
                     'Omicron_22B_BA5':Omicron_22B_BA5
                    